In [1]:
import pandas as pd
from typing import List
from sqlalchemy.orm import joinedload

from app.llm.interface import LLMInterface
from app.core.task.utils import describe_goal
from app.config.database import SessionLocal
from app.storage.models import Task, TaskStatus
from notebooks.plan_mcts_optimizer import get_task_commit_tree

eval_client = LLMInterface("openai", "o3-mini")

def get_answer_commits(root_commit, commit_tree) -> List:
    if not root_commit.get("children"):
        final_answer = root_commit.get("vm_state", {}).get("variables", {}).get("final_answer", None)
        if final_answer:
            return [{
                "commit_hash": root_commit["commit_hash"],
                "committed_at": root_commit["committed_at"],
                "description": root_commit["description"],
                "seq_no": root_commit["seq_no"],
                "commit_type": root_commit["commit_type"],
                "parent_hash": root_commit["parent_hash"],
                "final_answer": final_answer,
                "reasoning": root_commit.get("vm_state", {}).get("reasoning", None),
                "plan": root_commit.get("vm_state", {}).get("current_plan", None),
            }]
        else:
            return []

    leaves = []
    for child_hash in root_commit.get("children"):
        leaves.extend(get_answer_commits(commit_tree.get(child_hash), commit_tree))
    return leaves

def find_plan_commit(plan, answers):
    for answer in answers:
        if answer['plan'] == plan:
            return answer
    return None


def list_tasks(filter_out_ids=None):
    with SessionLocal() as session:
        query = (
            session.query(Task)
            .options(joinedload(Task.namespace))
            .filter(Task.status != TaskStatus.deleted)
            .filter(Task.best_plan != None)
        )
        
        # Add filter only if we have IDs to exclude
        if filter_out_ids:
            query = query.filter(Task.id.not_in(filter_out_ids))
            
        return query.order_by(Task.updated_at.desc()).all()

In [2]:
import os

columns = ['id', 'goal', 'metadata', 'reasoning', 'plan', 
           'plan_commit_hash', 'answers', 'qualified_answers', 
           'is_valid', 'language']
local_sample_df = pd.DataFrame(columns=columns)


local_sample_file = "notebooks/local_samples.pkl"
try:
    if os.path.exists(local_sample_file):
        local_sample_df = pd.read_pickle(local_sample_file)
    task_ids_list = local_sample_df['id'].tolist()
    tasks = list_tasks(task_ids_list)
except Exception as e:
    print(f"Error reading local sample file: {e}")
    tasks = []

In [ ]:
samples = []
index = 0

for task in tasks:
    index += 1
    print(index, "Processing task to get plan samples", task.id)
    commit_tree = get_task_commit_tree(task.id)
    if len(commit_tree) == 0:
        print(f"No commit tree found for task {task.id}, skipping")
        metadata = task.meta
        response_format = metadata.get("response_format", {})
        sample = {
            "id": task.id,
            "goal": task.goal,
            "metadata": metadata,
            "plan": task.best_plan,
            "reasoning": None,
            "final_answer": None,
            "plan_commit_hash": None,
            "answers": [],
            "qualified_answers": [],
            "is_valid": True,
            "language": response_format.get("Lang") or response_format.get("lang") if response_format else None
        }
        print(sample)
        samples.append(sample)
        continue
    try:
        root_commit = next(
            commit for _, commit in commit_tree.items() if not commit["parent_hash"]
        )
    except StopIteration:
        print(f"No root commit found for task {task.id}")
        continue

    leaves = get_answer_commits(root_commit, commit_tree)
    try:
        metadata = task.meta
        answer_commit = find_plan_commit(task.best_plan, leaves)
        response_format = metadata.get("response_format", {})
        sample = {
            "id": task.id,
            "goal": task.goal,
            "metadata": metadata,
            "plan": task.best_plan,
            "reasoning": answer_commit.get("reasoning") if answer_commit else None,
            "final_answer": answer_commit.get("final_answer") if answer_commit else None,
            "plan_commit_hash": answer_commit.get("commit_hash") if answer_commit else None,
            "answers": leaves,
            "qualified_answers": [],
            "is_valid": True,
            "language": response_format.get("Lang") or response_format.get("lang") if response_format else None
        }
        print(sample)
        samples.append(sample)
    except Exception as e:
        print(f"Error describing goal for task {task.id}: {e}. Data {task.meta}")
        raise e


In [ ]:
new_samples_df = pd.DataFrame(samples)
local_sample_df = pd.concat([local_sample_df, new_samples_df], ignore_index=True)

# Save updated DataFrame back to file
local_sample_df.to_pickle(local_sample_file)

print(f"Added {len(samples)} new samples. Total samples: {len(local_sample_df)}")

In [9]:
from app.utils.json import extract_json

def _evaluate_criteria(llm_client, prompt_template: str, context: dict) -> tuple[int, str]:
    """Evaluate a single quality criteria using LLM.
    
    Args:
        llm_client: Initialized LLM client
        prompt_template: Prompt template with placeholders
        context: Dictionary containing template variables
        
    Returns:
        Tuple of (score, explanation) with fail-safes for errors
    """
    try:
        formatted_prompt = prompt_template.format(**context)
        response = llm_client.generate(formatted_prompt)
        json_obj = extract_json(response)
        return json_obj.get("score", 0), json_obj.get("explanation", "No explanation provided")
    except Exception as e:
        print(f"Evaluation failed: {str(e)}, data: {response}")
        raise e

def evaluate_and_filter_answers(llm_client, row, threshold: int = 7) -> list[dict]:
    """Evaluate answers against quality criteria and filter based on scores.
    
    Args:
        llm_client: Initialized LLM client interface
        row: DataFrame row containing goal and answers
        threshold: Minimum score required for retention (0-10 scale)
        
    Returns:
        List of filtered answer dictionaries with evaluation metadata
    """
    filtered_answers = []
    goal_desc = describe_goal(row['goal'], row['metadata'])
    
    for answer in row['answers']:
        print(f"Evaluating answer: {answer['commit_hash']}")
        # Prepare evaluation context
        context = {
            'goal_desc': goal_desc,
            'final_answer': answer.get('final_answer', ''),
            'plan': str(answer.get('plan', '')),
            'response_lang': row['metadata'].get('response_language', 'match goal language')
        }

        # 1. Language Consistency Evaluation
        lang_prompt = """Evaluate language consistency between goal and answer:
Goal: {goal_desc} (Response Language: {response_lang})
Answer: {final_answer}

Checklist:
✓ Language matches specified response language
✓ Consistent terminology usage
✓ No unintentional code-switching
✓ Grammar/syntax correctness

Provide your evaluation in this JSON format:

{{
    "score": [0-10], # 10 is the highest score means the plan and answer are perfect.
    "explanation": "Detailed explanation of the score..."
}}"""

        lang_score, lang_expl = _evaluate_criteria(llm_client, lang_prompt, context)

        # 2. Logical Flow Evaluation  
        logic_prompt = """Analyze logical consistency of this plan for '{goal_desc}':
Plan: {plan}

Checklist:
✓ Step-by-step progression without gaps
✓ No contradictory statements
✓ Clear termination condition
✓ Appropriate error handling
✓ Unnecessary repeated steps (Note: Different tools for the same query are intentional design choices and should NOT be considered redundant.)
✓ Redundant variables
✓ Over-engineering indicators

Note: Different query methods for the same target are intentional design choices and should NOT be considered redundant.


Provide your evaluation in this JSON format:

{{
    "score": [0-10], # 10 is the highest score means the plan is perfect.
    "explanation": "Detailed explanation of the score..."
}}"""
        logic_score, logic_expl = _evaluate_criteria(llm_client, logic_prompt, context)

        # Store evaluation metadata
        evaluation_res = {
            'lang_score': lang_score,
            'lang_explanation': lang_expl,
            'logic_score': logic_score,
            'logic_explanation': logic_expl,
        }
        answer.update(evaluation_res)
        if all(score >= threshold for score in [lang_score, logic_score]):
            filtered_answers.append(answer)
        else:
            print(f"Answer {answer['commit_hash']} failed one or more checks:{evaluation_res}")
    
    return filtered_answers

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

# Evaluate unevaluated samples
for index, row in tqdm(local_sample_df.iterrows(), total=len(local_sample_df), desc="Evaluating answers"):
    # Skip if already evaluated (qualified_answers is not empty)
    if row['qualified_answers'] and len(row['qualified_answers']) > 0:
        continue

    if row['answers'] is None or len(row['answers']) == 0:
        continue
        
    try:
        # Evaluate and update qualified_answers
        qualified_answers = evaluate_and_filter_answers(eval_client, row)
        local_sample_df.at[index, 'qualified_answers'] = qualified_answers
        local_sample_df.at[index, 'is_valid'] = len(qualified_answers) > 0 and row['plan_commit_hash'] in [ans['commit_hash'] for ans in qualified_answers]
        
        # Save periodically (e.g., every 10 samples)
        if index % 10 == 0:
            local_sample_df.to_pickle(local_sample_file)
            
    except Exception as e:
        print(f"Error processing row {index}: {str(e)}")
        continue

# Save final results
local_sample_df.to_pickle(local_sample_file)

# Print summary
total_evaluated = len(local_sample_df[local_sample_df['is_valid'] == True])
print(f"Evaluation complete. {total_evaluated} samples are valid (have qualified answers and valid best plan).")

In [49]:
def evaluate_best_plan(llm_client, row) -> str:
    """
    Evaluate and select the best plan from filtered answers using LLM.
    Returns the commit hash of the best plan.
    """
    if not row['filtered_answers']:
        return None
    
    # Prepare evaluation prompt
    prompt = f"""Evaluate these plan candidates for the goal: {row['goal_description']}

Evaluation Criteria:
1. Answer Quality: Correctness, completeness, and clarity of final answer
2. Plan Logic: Logical flow, step progression, and error handling
3. Efficiency: Minimal redundant steps while maintaining intentional design choices
4. Documentation: Clear reasoning and plan documentation

Plans to Evaluate:
"""
    for idx, answer in enumerate(row['filtered_answers'], 1):
        prompt += f"\nPlan {idx} (Commit: {answer['commit_hash']}):\n"
        prompt += f"- Final Answer: {answer['final_answer']}\n"
        prompt += f"- Plan: {answer['plan']}\n"
        prompt += f"- Reasoning: {answer['reasoning']}\n"

    prompt += """
Output MUST be JSON format:
{
    "best_commit": "commit_hash",
    "explanation": "Detailed comparison analysis..."
}"""

    try:
        response = llm_client.generate(prompt)
        result = extract_json(response)
        return result.get('best_commit')
    except Exception as e:
        print(f"Best plan evaluation failed for task {row['task_id']}: {str(e)}")
        return None

In [29]:

def generate_reasoning(llm_client, row):
    goal = row['goal']
    best_plan = row['plan']
    final_answer = None
    for answer in row['answers']:
        if row['plan_commit_hash'] == answer['commit_hash']:
            final_answer = answer['final_answer']
            break
    
    prompt = (
        'You are tasked with generating a reasoning process based on a given goal, which should lead to a specific plan and final answer. However, the reasoning must represent the detailed thought process that occurs *before* the plan is created and the final answer is determined. Here\'s the information:\n\n'
        f'Goal: "{goal}"\n'
        f'Reference Plan (do not mention in output): "{best_plan}"\n'
        f'Reference Final Answer (do not mention in output): "{final_answer}"\n\n'
        'Your task is to write a reasoning narrative that:\n'
        '- Starts with the goal and explores how to approach it.\n'
        '- Includes logical, detailed steps of thinking, such as identifying what needs to be done, breaking it into smaller sub-goals, considering possible methods or tools, reflecting on potential challenges or alternatives, and systematically narrowing down the approach.\n'
        '- Uses a natural, human-like thought process (e.g., "I need to... First, I\'ll... Then considering... What if... Maybe I should also check... This makes sense because... Therefore...").\n'
        '- Does NOT explicitly mention the specific steps of the reference plan or the reference final answer, but ensures the reasoning logically and naturally builds toward them.\n'
        '- Remains a general thought process that could precede the creation of the plan and answer.\n'
        '- Must be detailed and thorough, providing a rich exploration of the problem with high-quality insights (e.g., justifying choices, weighing pros and cons, considering edge cases), while avoiding irrelevant tangents or redundant filler.\n'
        '- Should feel like a coherent, step-by-step intellectual journey that demonstrates deep engagement with the goal.\n\n'
        'Output the reasoning as a concise yet detailed paragraph or a short sequence of thoughts. Here\'s an example for reference:\n\n'
        'Example Goal: "Find the value of x in the equation x + 3 = 7"\n'
        'Example Reference Plan: "Step 1: Subtract 3 from both sides. Step 2: Simplify to find x."\n'
        'Example Reference Final Answer: "4"\n'
        'Example Reasoning: "I need to figure out what x equals in this equation. Looking at it, I see x is being added to 3, and that total is 7. My goal is to isolate x, so I need to think about how to get rid of the 3. What’s happening here mathematically? Addition is tying x and 3 together, so maybe I can reverse that operation. If I subtract something, though, won’t that change the equation? Wait—I remember equations need balance, so if I adjust one side, I have to adjust the other too. Let’s explore: if I take 3 away from 7, I’d get a number, but I need x alone on the other side. So, subtracting 3 from both sides might work. Could there be another way, like multiplying or dividing? No, that doesn’t fit here—addition’s the key. This feels promising; I’ll proceed by simplifying after balancing both sides, and that should reveal x."\n\n'
        f'Now, generate the reasoning for the given goal: "{goal}", ensuring it aligns with the reference plan "{best_plan}" and reference final answer "{final_answer}" without mentioning their specific details. Provide a detailed, high-quality thought process that deeply engages with the goal.'
    )

    if final_answer is None:
        prompt = (
            'You are tasked with generating a reasoning process based on a given goal, which should lead to a specific plan. However, the reasoning must represent the detailed thought process that occurs *before* the plan is created. Here\'s the information:\n\n'
            f'Goal: "{goal}"\n'
            f'Reference Plan (do not mention in output): "{best_plan}"\n'
            'Your task is to write a reasoning narrative that:\n'
            '- Starts with the goal and explores how to approach it.\n'
            '- Includes logical, detailed steps of thinking, such as identifying what needs to be done, breaking it into smaller sub-goals, considering possible methods or tools, reflecting on potential challenges or alternatives, and systematically narrowing down the approach.\n'
            '- Uses a natural, human-like thought process (e.g., "I need to... First, I\'ll... Then considering... What if... Maybe I should also check... This makes sense because... Therefore...").\n'
            '- Does NOT explicitly mention the specific steps of the reference plan, but ensures the reasoning logically and naturally builds toward them.\n'
            '- Remains a general thought process that could precede the creation of the plan and answer.\n'
            '- Must be detailed and thorough, providing a rich exploration of the problem with high-quality insights (e.g., justifying choices, weighing pros and cons, considering edge cases), while avoiding irrelevant tangents or redundant filler.\n'
            '- Should feel like a coherent, step-by-step intellectual journey that demonstrates deep engagement with the goal.\n\n'
            'Output the reasoning as a concise yet detailed paragraph or a short sequence of thoughts. Here\'s an example for reference:\n\n'
            'Example Goal: "Find the value of x in the equation x + 3 = 7"\n'
            'Example Reference Plan: "Step 1: Subtract 3 from both sides. Step 2: Simplify to find x."\n'
            'Example Reasoning: "I need to figure out what x equals in this equation. Looking at it, I see x is being added to 3, and that total is 7. My goal is to isolate x, so I need to think about how to get rid of the 3. What’s happening here mathematically? Addition is tying x and 3 together, so maybe I can reverse that operation. If I subtract something, though, won’t that change the equation? Wait—I remember equations need balance, so if I adjust one side, I have to adjust the other too. Let’s explore: if I take 3 away from 7, I’d get a number, but I need x alone on the other side. So, subtracting 3 from both sides might work. Could there be another way, like multiplying or dividing? No, that doesn’t fit here—addition’s the key. This feels promising; I’ll proceed by simplifying after balancing both sides, and that should reveal x."\n\n'
            f'Now, generate the reasoning for the given goal: "{goal}", ensuring it aligns with the reference plan "{best_plan}"" without mentioning their specific details. Provide a detailed, high-quality thought process that deeply engages with the goal.'
        )
    
    reasoning = llm_client.generate(prompt)
    return reasoning


In [ ]:
from tqdm.auto import tqdm

# Process each row to generate missing reasoning
for index, row in tqdm(local_sample_df.iterrows(), total=len(local_sample_df), desc="Generating reasoning"):
    # Skip if reasoning already exists and is not empty
    if row['reasoning'] and isinstance(row['reasoning'], str) and len(row['reasoning'].strip()) > 0:
        continue
        
    try:
        # Generate reasoning for this row
        reasoning = generate_reasoning(eval_client, row)
        local_sample_df.at[index, 'reasoning'] = reasoning
        
        # Save periodically to prevent data loss
        if index % 10 == 0:
            local_sample_df.to_pickle(local_sample_file)
            
    except Exception as e:
        print(f"Error generating reasoning for row {index}: {str(e)}")
        continue

# Save final results
local_sample_df.to_pickle(local_sample_file)

# Print summary
total_with_reasoning = len(local_sample_df[local_sample_df['reasoning'].notna()])
print(f"Processing complete. {total_with_reasoning} samples now have reasoning.")

In [3]:
def adjust_plan_steps(plan):
    """
    remove useless steps
    """
    if not plan or not isinstance(plan, list):
        return plan
    
    origin_length = len(plan)
    
    # deep copy to avoid modifying original data
    new_plan = [step.copy() for step in plan if isinstance(step, dict)]
    
    # check the first step
    if new_plan and new_plan[0].get('type') == 'reasoning' and new_plan[0].get('seq_no') == 0:
        # delete the first step
        new_plan.pop(0)
        
        # re-number the remaining steps
        for idx, step in enumerate(new_plan):
            step['seq_no'] = idx

        if len(new_plan) != origin_length - 1:
            raise ValueError(f"The number of steps in the adjusted plan does not match the original plan: {len(new_plan)} != {origin_length - 1}")
        
        expected_seq = 0
        for step in new_plan:
            if step.get('seq_no') != expected_seq:
                raise ValueError(f"The sequence of steps in the adjusted plan is not continuous: {step.get('seq_no')} != {expected_seq}")
            expected_seq += 1
        return new_plan
    else:
        if len(new_plan) != origin_length:
            raise ValueError(f"The number of steps in the adjusted plan does not match the original plan: {len(new_plan)} != {origin_length}")
    
    return new_plan

In [4]:
from tqdm.auto import tqdm

# Process each row to adjust plans
for index, row in tqdm(local_sample_df.iterrows(), total=len(local_sample_df), desc="Adjusting plans"):
    try:
        # Adjust plan for this row
        new_plan = adjust_plan_steps(row['plan'])
        local_sample_df.at[index, 'plan'] = new_plan
    except Exception as e:
        print(f"Error adjusting plan for row {index}: {str(e)}")
        continue

/Users/ianz/miniconda3/envs/stackvm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Adjusting plans: 100%|██████████| 175/175 [00:00<00:00, 14507.14it/s]


In [8]:
selected_columns = ['id','goal', 'metadata', 'reasoning', 'plan']
dataset_df =   local_sample_df[selected_columns].copy()

In [9]:
dataset_df.columns


Index(['id', 'goal', 'metadata', 'reasoning', 'plan'], dtype='object')

In [14]:
import json
from datasets import Dataset


for col in ["metadata", "plan"]:
    dataset_df[col] = dataset_df[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else x)

dataset_df = dataset_df.reset_index(drop=True)
dataset = Dataset.from_pandas(dataset_df)
dataset


Dataset({
    features: ['id', 'goal', 'metadata', 'reasoning', 'plan'],
    num_rows: 175
})

In [16]:
dataset.push_to_hub("ianthereal-z/tidb_bot")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.98s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ianthereal-z/tidb_bot/commit/3796106600b99d00bec693b4d881fedd3093d9d7', commit_message='Upload dataset', commit_description='', oid='3796106600b99d00bec693b4d881fedd3093d9d7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ianthereal-z/tidb_bot', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ianthereal-z/tidb_bot'), pr_revision=None, pr_num=None)